In [8]:
import platform
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import sklearn
from sklearn import datasets, decomposition, metrics, preprocessing, utils
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score


In [13]:
DATA = Path("data")
assert DATA.exists()

sncf_data = pd.read_csv(DATA / "sncf_data.csv", low_memory=False, on_bad_lines="skip",delimiter=";")
sncf_data

,Date,Service,Gare de départ,Gare d'arrivée,Durée moyenne du trajet,Nombre de circulations prévues,Nombre de trains annulés,Commentaire annulations,Nombre de trains en retard au départ,Retard moyen des trains en retard au départ,...,Nombre trains en retard > 15min,Retard moyen trains en retard > 15 (si liaison concurrencée par vol),Nombre trains en retard > 30min,Nombre trains en retard > 60min,Prct retard pour causes externes,Prct retard pour cause infrastructure,Prct retard pour cause gestion trafic,Prct retard pour cause matériel roulant,Prct retard pour cause gestion en gare et réutilisation de matériel,"Prct retard pour cause prise en compte voyageurs (affluence, gestions PSH, correspondances)"
0,2018-01,National,BORDEAUX ST JEAN,PARIS MONTPARNASSE,141,870,5,NaN,289,11.247809,...,110,6.511118,44,8,36.134454,31.092437,10.924370,15.966387,5.042017,0.840336
1,2018-01,National,LA ROCHELLE VILLE,PARIS MONTPARNASSE,165,222,0,NaN,8,2.875000,...,22,5.696096,5,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000
2,2018-01,National,PARIS MONTPARNASSE,QUIMPER,220,248,1,NaN,37,9.501351,...,26,7.548387,17,7,26.923077,38.461538,15.384615,19.230769,0.000000,0.000000
3,2018-01,National,PARIS MONTPARNASSE,ST MALO,156,102,0,NaN,12,19.912500,...,8,6.724757,6,4,23.076923,46.153846,7.692308,15.384615,7.692308,0.000000
4,2018-01,National,PARIS MONTPARNASSE,ST PIERRE DES CORPS,61,391,2,NaN,61,7.796995,...,17,3.346487,6,0,21.212121,42.424242,9.090909,21.212121,6.060606,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9956,2024-09,National,PARIS MONTPARNASSE,QUIMPER,225,265,2,NaN,37,24.755856,...,25,52.180000,15,6,24.000000,16.000000,8.000000,24.000000,12.000000,16.000000
9957,2024-09,National,PARIS MONTPARNASSE,RENNES,97,780,2,NaN,96,19.672222,...,51,52.480719,22,13,7.777778,21.111111,13.333333,25.555556,21.111111,11.111111
9958,2024-09,National,PARIS MONTPARNASSE,TOULOUSE MATABIAU,269,214,0,NaN,30,9.938889,...,40,46.442500,19,7,15.000000,35.000000,15.000000,17.500000,7.500000,10.000000
9959,2024-09,National,TOULOUSE MATABIAU,PARIS MONTPARNASSE,275,215,0,NaN,21,21.730159,...,30,59.802778,21,12,23.333333,23.333333,10.000000,6.666667,16.666667,20.000000


In [10]:
X=sncf_data[["Date","Gare de départ","Gare d'arrivée","Durée moyenne du trajet","Nombre de circulations prévues"]] 
Y_nb_retard_depart=sncf_data[["Nombre de trains en retard au départ"]]
Y_retard_moy_depart = sncf_data[["Retard moyen de tous les trains au départ"]]
Y_nb_retard_arrivee=sncf_data[["Nombre de trains en retard à l'arrivée"]]
Y_retard_moy_arrivee=sncf_data[["Retard moyen de tous les trains à l'arrivée"]]
Y= [Y_nb_retard_depart,Y_retard_moy_depart,Y_nb_retard_arrivee,Y_retard_moy_arrivee]

In [11]:
X_train_tab = []
X_test_tab = []
y_train_tab = []
y_test_tab =  []

In [12]:
model = PolynomialFeatures(degree=5)
X_train, X_test, y_train, y_test = train_test_split(X,Y_nb_retard_arrivee,test_size=0.2,random_state=42)
X_train_tf = model.fit_transform(X_train)
X_test_tf = model.fit_transform(X_test)

model_rl = LinearRegression()
model_rl.fit(X_train_tf, y_train)
y_pred = model_rl.predict(X_test_tf)

mse = mean_squared_error(X_test_tf,y_pred)
print("Mean Squared Error:",
 mse)
r2 = r2_score(X_test,y_pred)
print("R-squared Score:", r2)

plt.figure(figsize=(10, 6))
plt.scatter(X_test,y_test,label='Original data')
plt.scatter(X_test,y_pred,color='red',label='prediction')
plt.legend()
plt.plot()

ValueError: could not convert string to float: '2023-01'